In [1]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

2023-12-01 03:13:35.285502: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-01 03:13:35.309578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 03:13:35.653952: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# TEST ENVIRONMENT

In [2]:
#!wget https://www.atarimania.com/roms/Roms.rar

In [3]:
!python3 -m atari_py.import_roms ./ROMS

copying carnival.bin from ./ROMS/Carnival (1982) (Coleco - Woodside Design Associates, Steve 'Jessica Stevens' Kitchen) (2468) ~.bin to /home/slv/anaconda3/envs/rl/lib/python3.10/site-packages/atari_py/atari_roms/carnival.bin
copying ice_hockey.bin from ./ROMS/Ice Hockey - Le Hockey Sur Glace (1981) (Activision, Alan Miller) (AX-012, CAX-012, AX-012-04) ~.bin to /home/slv/anaconda3/envs/rl/lib/python3.10/site-packages/atari_py/atari_roms/ice_hockey.bin
copying pong.bin from ./ROMS/Video Olympics - Pong Sports (Paddle) (1977) (Atari, Joe Decuir - Sears) (CX2621 - 99806, 6-99806, 49-75104) ~.bin to /home/slv/anaconda3/envs/rl/lib/python3.10/site-packages/atari_py/atari_roms/pong.bin
copying atlantis.bin from ./ROMS/Atlantis (Lost City of Atlantis) (1982) (Imagic, Dennis Koble) (720103-1A, 720103-1B, IA3203, IX-010-04) ~.bin to /home/slv/anaconda3/envs/rl/lib/python3.10/site-packages/atari_py/atari_roms/atlantis.bin
copying name_this_game.bin from ./ROMS/Name This Game (Guardians of Treas

In [4]:
environment_name='Breakout-v0'
env=gym.make(environment_name,render_mode='rgb_array')

/home/slv/anaconda3/envs/rl/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [5]:
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {'lives': 5, 'episode_frame_number': 0, 'frame_number': 0})

In [6]:
env.step(env.action_space.sample())

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 0.0,
 False,
 False,
 {'lives': 5, 'episode_frame_number': 2, 'frame_number': 2})

In [7]:
env.action_space

Discrete(4)

In [8]:
env.action_space.sample()

0

In [9]:
env.observation_space
#hright,width,rgb

Box(0, 255, (210, 160, 3), uint8)

In [10]:
from render_browser import render_browser

@render_browser
def test_env(model):
    episodes=5
    for episode in range(1,episodes+1):
        obs=env.reset
        done = False
        score=0

        while not done:
            yield env.render()
            action = env.action_space.sample()
            obs,reward,terminated,truncated,info=env.step(action)
            done= terminated or truncated
            score+=reward
        print(f'Episode: {episode} , Score: {score}')
    env.close()

    

In [11]:
#test_env("")

In [17]:
env=make_atari_env('Breakout-v0',n_envs=4,seed=0)
# to write wrapped atari environments
env=VecFrameStack(env,n_stack=4)
env.reset()



/home/slv/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


array([[[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        ...,

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]],


       [[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0

In [18]:
log_path=os.path.join('Training','Logs')
model=A2C('CnnPolicy',env,verbose=1,tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [19]:
model.learn(total_timesteps=1000000)

Logging to Training/Logs/A2C_3


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 289      |
|    ep_rew_mean        | 1.78     |
| time/                 |          |
|    fps                | 654      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | -0.195   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.093   |
|    value_loss         | 0.0128   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 273      |
|    ep_rew_mean        | 1.42     |
| time/                 |          |
|    fps                | 650      |
|    iterations         | 200      |
|    time_elapsed       | 6        |
|    total_timesteps    | 4000     |
| train/                |          |
|

In [20]:
model_path=os.path.join("Training","SavedModels","A2C_Breakout_1M")
model.save(model_path)

# EVALUATE

In [21]:
env=make_atari_env('Breakout-v0',n_envs=1,seed=0)
# to write wrapped atari environments
env=VecFrameStack(env,n_stack=16)
evaluate_policy(model,env,n_eval_episodes=10,render=False)

ValueError: Error: Unexpected observation shape (1, 84, 84, 16) for Box environment, please use (4, 84, 84) or (n_env, 4, 84, 84) for the observation shape.